In [4]:
import torch
import tensorly as tl
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import QuantLib as ql
import pysabr
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
from scipy.optimize import minimize

%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

In [5]:
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F

from tensorly.tenalg import inner

In [6]:
tl.set_backend('pytorch')
batch_size = 64
device = 'cpu'
kf = KFold(n_splits=4, shuffle=True)

In [8]:
scaler = MinMaxScaler()

raw_data = pd.read_csv('fdm_hagan_vol.csv')
test_raw_data = pd.read_csv('fdm_hagan_vol_test.csv')
raw_data["Error"] = (raw_data["FDM_vol"] - raw_data["Hagan_vol"]).abs()
test_raw_data["Error"] = (test_raw_data["FDM_vol"] - test_raw_data["Hagan_vol"]).abs()

X = raw_data.drop(["FDM_vol", "Hagan_vol", "Error"], axis=1).values
y = raw_data["Error"].values
X_test = test_raw_data.drop(["FDM_vol", "Hagan_vol", "Error"], axis=1).values
y_test = test_raw_data["Error"].values

scaler.fit(X)
X = scaler.transform(X)
X_test = scaler.transform(X_test)


# for train_index, test_index in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

In [ ]:
# from sklearn import linear_model
# from sklearn import svm
# from sklearn.metrics import mean_squared_error

# classifiers = [
#     svm.SVR(),
#     linear_model.SGDRegressor(),
#     linear_model.BayesianRidge(),
#     linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.PassiveAggressiveRegressor(),
#     linear_model.TheilSenRegressor(),
#     linear_model.LinearRegression()]

# for item in classifiers:
#     print(item)
#     clf = item
#     clf.fit(X, y)
#     print(clf.predict(X_test),'\n')
#     print("RMSE: ",mean_squared_error(y_test, clf.predict(X_test), squared=False))

SVR()
[ 0.00075146 -0.04030514 -0.01026842 ...  0.00205002  0.00258336
  0.04109529] 

RMSE:  0.0327799483505655
SGDRegressor()
[ 0.08826189  0.00954725  0.04439383 ... -0.02209945 -0.0263771
  0.11485972] 

RMSE:  0.056796939410364435
BayesianRidge()
[ 0.08879069  0.01012903  0.04427379 ... -0.0222724  -0.026768
  0.11495062] 

RMSE:  0.05684879902113361
LassoLars()
[0.07139829 0.07139829 0.07139829 ... 0.07139829 0.07139829 0.07139829] 

RMSE:  0.10174129528515419
ARDRegression()
[ 0.08830367  0.01032178  0.04371237 ... -0.02229401 -0.02698608
  0.11395563] 

RMSE:  0.056854148967893786
PassiveAggressiveRegressor()
[ 0.06342781 -0.00552001  0.04139412 ... -0.0362765  -0.05068135
  0.0721878 ] 

RMSE:  0.06603461069169445
TheilSenRegressor(max_subpopulation=10000)


In [49]:
# logistic_regression_model = LogisticRegression()
# logistic_regression_model.fit(X, y)
# y_logreg_pred = logistic_regression_model.predict(X_test)

ValueError: Unknown label type: 'continuous'

In [38]:
X = torch.FloatTensor(X)
y = torch.FloatTensor(y)
y = torch.reshape(y, shape=(-1,1))

X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)
y_test = torch.reshape(y_test, shape=(-1,1))

In [39]:
train_dataset = torch.utils.data.TensorDataset(X, y)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [42]:
model = nn.Sequential(nn.Linear(6, 20, device=device),
                      nn.Linear(20,100),
                      nn.Linear(100,400),
                      nn.Sigmoid(),
                      nn.Linear(400,10),           
                      nn.Tanh(),
                      nn.Linear(10,1),
                      nn.Tanh()
)

optimizer = optim.Adam(model.parameters(),lr=0.001)
n_epochs = 20
# batch_size = 64

model = model.to(device)


In [45]:
def train(n_epoch):
    model.train()
    for i, (features, error) in enumerate(train_loader):
        features, error = features.to(device), error.to(device)
        optimizer.zero_grad()
        output = model(features)
        loss = torch.nn.functional.mse_loss(output,error)
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(features), len(train_loader.dataset),
                100. * i / len(train_loader), loss))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for features, error in test_loader:
        features, error = features.to(device), error.to(device)
        output = model(features)
        test_loss = torch.nn.functional.mse_loss(output,error)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(error.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('mean: {}'.format(test_loss))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))

for epoch in range(1, n_epochs):
    train(epoch)
    test()


Train Epoch: 1 [0/553320 (0%)]	Loss: 0.000013
Train Epoch: 1 [64000/553320 (12%)]	Loss: 0.000005
Train Epoch: 1 [128000/553320 (23%)]	Loss: 0.000009
Train Epoch: 1 [192000/553320 (35%)]	Loss: 0.000004
Train Epoch: 1 [256000/553320 (46%)]	Loss: 0.000034
Train Epoch: 1 [320000/553320 (58%)]	Loss: 0.000005
Train Epoch: 1 [384000/553320 (69%)]	Loss: 0.000004
Train Epoch: 1 [448000/553320 (81%)]	Loss: 0.000007
Train Epoch: 1 [512000/553320 (93%)]	Loss: 0.000001
mean: 2.072007843079149e-11

Test set: Average loss: 0.0000, Accuracy: 0/50000 (0%)

Train Epoch: 2 [0/553320 (0%)]	Loss: 0.000004
Train Epoch: 2 [64000/553320 (12%)]	Loss: 0.000005
Train Epoch: 2 [128000/553320 (23%)]	Loss: 0.000005
Train Epoch: 2 [192000/553320 (35%)]	Loss: 0.000004
Train Epoch: 2 [256000/553320 (46%)]	Loss: 0.000005
Train Epoch: 2 [320000/553320 (58%)]	Loss: 0.000003
Train Epoch: 2 [384000/553320 (69%)]	Loss: 0.000014
Train Epoch: 2 [448000/553320 (81%)]	Loss: 0.000002
Train Epoch: 2 [512000/553320 (93%)]	Loss: 0.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        
        self.trl = TRL(ranks=(10, 3, 3, 10), input_size=(batch_size, 50, 4, 4), output_size=(batch_size,10))

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.trl(x)
        return F.log_softmax(x)


model = Net()